In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
#torch.backends.cudnn.enabled = False


In [2]:
manualSeed=187
fsize=64
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

torch.cuda.manual_seed_all(manualSeed)

cudnn.benchmark = True

dataroot='batched_dataset_256/mat_files/'
path=dataroot

Random Seed:  187


In [3]:
import os
import scipy.io
ls=os.listdir(path)
#print ls

In [4]:
xs=[]
ys=[]
for i in ls:
    im1 = scipy.io.loadmat(path+i)
    im1=im1['im']
    xs.append(im1.shape[0])
    ys.append(im1.shape[1])

In [5]:
s1=min(xs)
s2=min(ys)
print (s1)
print (s2)

256
256


In [6]:
imageSize=(s1,s2,3)
lls=len(ls)
import numpy as np
X=np.zeros((lls,3,s1,s2))
Y=np.zeros((lls,3,s1,s2))
count=0
import Image
for idx,i in enumerate(ls):
    li=len(i)
    i=i[0:li-4]
    im1 = scipy.io.loadmat(path+i+'.mat')
    im2=np.asarray(Image.open('batched_dataset_256/image_files/'+i+'.png'))    
    im2=im2/255.0;
    im1=im1['im']

    X[count,0,:,:]=im1[ 0:s1    , 0:s2 ,0]
    X[count,1,:,:]=im1[ 0:s1    , 0:s2 ,1]
    X[count,2,:,:]=im1[ 0:s1    , 0:s2 ,2]

    Y[count,0, 0:s1    , 0:s2 ]=im2[ 0:s1    , 0:s2 ,0]
    Y[count,1, 0:s1    , 0:s2 ]=im2[ 0:s1    , 0:s2 ,1]
    Y[count,2, 0:s1    , 0:s2 ]=im2[ 0:s1    , 0:s2 ,2]
    count=count+1
    
 

In [7]:
   
print(X.shape)

(474, 3, 256, 256)


In [8]:
#time pass
def convert_channels(a):
    b=np.zeros((a.shape[1],a.shape[2],3))
    b[:,:,0]=a[0,:,:]
    b[:,:,1]=a[1,:,:]
    b[:,:,2]=a[2,:,:]
    return b   

def convert_channels2(a):
    b=np.zeros((3,a.shape[1],a.shape[2]))
    b[0,:,:]=a[:,:,0]
    b[1,:,:]=a[:,:,1]
    b[2,:,:]=a[:,:,2]
    return b   



In [9]:
'''
import scipy.io as sio
lx=X.shape[0]
imgs=np.zeros(X.shape)
for i in range(lx):
    sio.savemat(path+str(i)+'.mat',{'im':X[i,:,:,:]})
    a=X[i,:,:,:]
    im1_8bit = np.clip(a*255, 0, 255).astype('uint8')
    imgs[i,:,:,:]=( im1_8bit)
'''

"\nimport scipy.io as sio\nlx=X.shape[0]\nimgs=np.zeros(X.shape)\nfor i in range(lx):\n    sio.savemat(path+str(i)+'.mat',{'im':X[i,:,:,:]})\n    a=X[i,:,:,:]\n    im1_8bit = np.clip(a*255, 0, 255).astype('uint8')\n    imgs[i,:,:,:]=( im1_8bit)\n"

In [10]:
'''
# save the training set
path='batched_hdr/'
path2='batched_hdr_tone/'
import scipy.io as sio
lx=X.shape[0]
for i in range(lx):
    sio.savemat(path+str(i)+'.mat',{'im':X[i,:,:,:]})
    see_image(X[i,:,:,:],i,path2)
'''

"\n# save the training set\npath='batched_hdr/'\npath2='batched_hdr_tone/'\nimport scipy.io as sio\nlx=X.shape[0]\nfor i in range(lx):\n    sio.savemat(path+str(i)+'.mat',{'im':X[i,:,:,:]})\n    see_image(X[i,:,:,:],i,path2)\n"

In [11]:
print (X.shape)
#Z=np.zeros((X.shape[0],3,X.shape[1],X.shape[2]))
#Z[:,0,:,:]=X[:,:,:,0]
#Z[:,1,:,:]=X[:,:,:,1]
#Z[:,2,:,:]=X[:,:,:,2]
#X=Z

(474, 3, 256, 256)


In [12]:
batchSize=10
workers=1
def get_batch():
    import numpy.random
    pos=numpy.random.randint(0,X.shape[0],(batchSize))
    x1=X[pos]
    y1=Y[pos]
    return (x1), (y1)



ngpu = 1
nz = 100
ngf = 4
ndf = 4
nc = 3

In [13]:
#for i in dataloader:
#    print (i)

In [14]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [99]:
#class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
#class torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1)
class _netG(nn.Module):
    def __init__(self, ngpu):
        super(_netG, self).__init__()
        self.ngpu = ngpu
        model=[]
        xx=[]
        #model.append(nn.Conv2d(nc, ndf, 3, 2, 0, bias=False))
        xx.append(nn.Conv2d(nc, ndf, 3, 2, 0, bias=False))
        self.v1=64

        model.append(nn.LeakyReLU(0.2, inplace=True) )
        model.append(nn.Conv2d(ndf, ndf * 2, 5, 2, 3, bias=False))
        #self.bn1=nn.BatchNorm2d(ndf * 2)
        model.append(nn.LeakyReLU(0.2, inplace=True))
        model.append(nn.Conv2d(ndf * 2, ndf*4, 5, 2, 1, bias=False))
        #self.bn2=nn.BatchNorm2d(ndf * 4)
        model.append(nn.LeakyReLU(0.2, inplace=True))
        model.append(nn.Conv2d(ndf*4, 3, 5, 2, 3, bias=False))
        model.append(nn.Sigmoid())

        model2=[]
        model2.append(nn.ConvTranspose2d(    3, ngf * 8, 4, 2, 1, bias=False))
        model2.append(nn.BatchNorm2d(ngf * 8))
        model2.append(nn.ReLU(True))
        model2.append(   nn.ConvTranspose2d(ngf * 8, ngf * 2, 3, 2, 1, bias=False))
        model2.append(   nn.BatchNorm2d(ngf * 2))
        model2.append(  nn.ReLU(True))
        # state size. (ngf*2) x 16 x 16
        model2.append(   nn.ConvTranspose2d(ngf * 2,     ngf, 3, 2, 1, bias=False))
        model2.append(   nn.BatchNorm2d(ngf))
        model2.append(  nn.ReLU(True))
        
        
        # state size. (ngf) x 32 x 32
        model2.append(  nn.ConvTranspose2d(    ngf,      nc, 3, 2, 1, bias=False))
        model2.append(  nn.Tanh())
        self.model = nn.Sequential(*model)
        self.xx = nn.Sequential(*xx)
        self.model2 = nn.Sequential(*model2)
        

        
    def forward(self, input):
        #print (input)
        op=self.xx(input)
        output = self.model(op)
        #print (output.size())
        #vap=(output.size())
        #nnvp=vap[0]*vap[1]*vap[2]*vap[3]
    

        #output = output.view(-1, nnvp)        
        #        print (output,1)

        #output=output.transpose(0,1)
        #print (output,2)
        #va=(input.size())
        #print (va)
        #print (output.size())
        #print (nnvp)
        #print (va[0])

        #output=output.resize(va[0],int(nnvp/va[0]),1,1)
        #      print (output,3)
        #self.v1=int(nnvp/va[0])
        #print (self.v1)

        output=self.model2(output)

        return output


In [100]:
netG = _netG(ngpu)
netG.apply(weights_init)
str_netG=''
#str_netG='netG_epoch_1.pth'
if str_netG != '':
    netG.load_state_dict(torch.load(str_netG))
print(netG)

_netG (
  (model): Sequential (
    (0): LeakyReLU (0.2, inplace)
    (1): Conv2d(4, 8, kernel_size=(5, 5), stride=(2, 2), padding=(3, 3), bias=False)
    (2): LeakyReLU (0.2, inplace)
    (3): Conv2d(8, 16, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1), bias=False)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(16, 3, kernel_size=(5, 5), stride=(2, 2), padding=(3, 3), bias=False)
    (6): Sigmoid ()
  )
  (xx): Sequential (
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(2, 2), bias=False)
  )
  (model2): Sequential (
    (0): ConvTranspose2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(32, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(8, 4, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (7): 

In [101]:
cuda=True
lr=0.0002
beta1=0.5
niter=800000
outf='output_256'
criterion = nn.MSELoss()

input = torch.FloatTensor(batchSize, 3, imageSize[0], imageSize[1])

netG.cuda()
criterion.cuda()
input = input.cuda()
input = Variable(input)
criterion=nn.MSELoss()
criterion.cuda()
ss=Y.shape[0]
# setup optimizer
optimizer = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
import torch.nn
for epoch in range(niter):
    #print (epoch,1)
    data, target = get_batch()
    #print (epoch,1)

    data, target= torch.from_numpy(data), torch.from_numpy(target)
    data, target = data.float().cuda(), target.float().cuda()
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    #print (data)
    output = netG(data)
    #print (output)
    #print (target)

    #print (output.size())
    #print(target.size())
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    #print (epoch,2)
    if epoch % 300 == 0:
        xyvap=np.random.randint(ss)
        #vutils.save_image(Y[xyvap],
        #        '%s/real_samples.png' % outf)

        ddv=X[xyvap:xyvap+1]
        #print (ddv.shape)
        ddv2=Variable(torch.from_numpy(ddv).float().cuda())
        
        ddv3=Y[xyvap:xyvap+1]
        #print (ddv.shape)
        ddv4=Variable(torch.from_numpy(ddv3).float().cuda())
        #print (ddv2)
        fake = netG(ddv2)
        #print (fake.data)
        #print (ddv4)

        vutils.save_image(fake.data,
                '%s/samples_epoch_%03d.png' % (outf, epoch) , normalize=False)
        vutils.save_image(ddv4.data,
                '%s/samples_epoch_%03d.png' % (outf, epoch+1), normalize=False)

    if(epoch%100==0):
        print(epoch,loss.data[0])
    if(epoch%1000==0): 
        torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outf, epoch))

RuntimeError: input and target have different number of elements: input[10 x 3 x 265 x 265] has 2106750 elements, while target[10 x 3 x 256 x 256] has 1966080 elements at /b/wheel/pytorch-src/torch/lib/THCUNN/generic/MSECriterion.cu:12